<a href="https://colab.research.google.com/github/6X10/advanced-statistical-data-analysis/blob/main/kNN_classifier_based_on_Minkowski_distance_of_order_p.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from tqdm.notebook import tqdm

import torch
import torchvision #에서 data 불러올 것
import torchvision.transforms as transforms

In [ ]:
# MNIST, CIFAR10, CIFAR100 (MNIST->CIFAR10, CIFAR100 으로 바꿔서 코드 그대로 사용하면 됨)
trainset = torchvision.datasets.MNIST(root='./datasets/', train=True,
                                        download=True, transform=transforms.ToTensor())

testset = torchvision.datasets.MNIST(root='./datasets/', train=False,
                                       download=True, transform=transforms.ToTensor())
#root='./datasets/' ; download 받는 directory
#transform=transforms.ToTensor() ; 별다른 transform하지 않겠음.

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./datasets/MNIST/raw/train-images-idx3-ubyte.gz to ./datasets/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./datasets/MNIST/raw/train-labels-idx1-ubyte.gz to ./datasets/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./datasets/MNIST/raw/t10k-images-idx3-ubyte.gz to ./datasets/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./datasets/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./datasets/MNIST/raw



In [ ]:
print(trainset)
#50K vs 10K // train vs valid (6 fold validation) BUT 3 fold cross validation할 거면 40K vs 20K로 쪼개면 됨.
classes = trainset.classes
print(len(classes)) #class 몇개 있는지

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./datasets/
    Split: Train
    StandardTransform
Transform: ToTensor()
10


In [ ]:
print(testset)

classes = testset.classes
print(len(classes))

Dataset MNIST
    Number of datapoints: 10000
    Root location: ./datasets/
    Split: Test
    StandardTransform
Transform: ToTensor()
10


In [ ]:
print(trainset[0]) #tensor 형태
print(trainset[0][0].shape) #channel, 1*28*28 dimension data인데 각 entry마다의 차이를 통해서 데이터 간 가까운 정도(거리) 계산. 1*28*28 shape

(tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000

In [ ]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=60000, #number of train data
                                          shuffle=True) #가져올 때 shuffling 하겠다
testloader = torch.utils.data.DataLoader(testset, batch_size=100, #실제로는 모두 활용해야해서 10000개 활용해야 하고, shuffle 필요 없지만, debugging할 때는 random sampled 된 test data 활용, 100개 shuffle해서. 실제로 데이터 분석할 때는 모든 데이터 활용해야. (실습이니까)
                                         shuffle=True)

#if nueron network할 때, batch_size 256,64 등..조금씩 들고오면 됨. but 이때는 train data 모두 사용할 것

In [ ]:
#trainloader에서 실제 데이터 가져오는 작업
dataiter = iter(trainloader)
tr_images, tr_labels = dataiter.next() #데이터 타입 pytorch tensor 형태

In [ ]:
tr_images, tr_labels = tr_images.numpy(), tr_labels.numpy() #for numpy 연산, numpy type으로 변환
print(tr_images.shape) #train data 60000만 개, grey scale channel 한 개, grey scale 가로*세로 = 28*28
print(tr_labels.shape)

(60000, 1, 28, 28)
(60000,)


In [ ]:
#load test data
dataiter = iter(testloader)
te_images, te_labels = dataiter.next()

In [ ]:
te_images, te_labels = te_images.numpy(), te_labels.numpy()
print(te_images.shape)
print(te_labels.shape)

(100, 1, 28, 28)
(100,)


In [ ]:
class kNN():
    def __init__(self):
        print("kNN initialized..") #할게 없음

    #엄밀히 따지면 KNN은 학습이 이뤄지는 부분이 없음. but train data를 모델이 들고는 있어야 하므로.
    #모델에게 들고 있는 training data 알려주는 부분 -> 앞으로 데이터 가져올 때마다 training data와 비교해서 k개의 가장 가까운 데이터 찾아서 voting하기 위해서
    def fit(self, X, Y):
        self.num_train_data = X.shape[0]
        #굳이 reshape할 필요는 없으나..calculate distance할 때 summation 한 번에 끝내버리려고
        #X.shape ; 60000*1*28*28 -> 60000*784 after reshape
        self.Xtr=X.reshape(self.num_train_data,-1) #60000만 개를 넣고, 나머지 dimension 뒤에 몰아넣겠다.
        self.Ytr=Y.reshape(self.num_train_data,-1) #(60000,1)
        print(self.Xtr.shape)
        print(self.Ytr.shape)

    #실제로 어떤 데이터 X(test data), K값, p(minkowski distance's order) 받는 함수
    def predict(self, X , k, p):
        # p값 이상하면 굳이 계산해야 하나 싶어서 예외처리
        if p <=0:
          raise Exception("Check p value!")
        elif p>= 1e+3:
          raise Exception("Check p value!")

        num_test_data=X.shape[0] #데이터 들어올 때 데이터 instance 하나가 들어올 수도 있지만 여러개일 수도 ㅇ. 입력하는 test data 여러개의 데이터로 되어 있으므로 데이터 몇 개인지 체크해서 활용.
        X=X.reshape(num_test_data,-1) #training data에 대해서 reshaping해준 것과 같이 testing data도 reshaping (다룰 데이터 dimension 맞춰줌)
        print(X.shape)

        # Initialize output matrix to zeros
        Ypred = np.zeros(num_test_data, dtype=self.Ytr.dtype) #입력하는 test data 여러개의 데이터로 되어 있으므로 그 개수 활용해서 각각의 데이터 어떤 prediction 나오는지 저장

        # Loop over input test images matrix X
        for i in tqdm(range(num_test_data)): #각각의 test data 하나가 들어올 때마다 distance의 sum(=784 dim에 대해서 각각의 entry의 distance 얼마나 되는지 minkowski 이용해서 계산)
            # calculate distance (predict할 test data instance 하나에 대해서 training data의 모든 녀석들과의 거리 계산)
            distances = np.power(np.sum(np.power(np.abs(self.Xtr-X[i]),p),-1),1/p) #training data와 test data의 i번째 간의 차이 구해서 abs value 취하고, p승 한 후에 다 더 하고, 전체에 1/p 승

            # choose k nearest neighbors
            topK = np.argsort(distances)[:k] #argsort:오름차순 정렬
            #print(topK)

            # labels of kNNs (1~k번째로 가까운 label들의 ㅣist)
            votes = self.Ytr[topK].tolist()

            # returns the most common vote in votes list
            res = max(votes, key=votes.count) #가장 count가 높은 label

            # Predicted y is the most commont vote
            Ypred[i] = res[0] #res가 list 형태로 나오므로 res[0] 사용

        return Ypred

In [ ]:
model = kNN() #우리의 모델을 KNN으로 instance화 함
model.fit(tr_images,tr_labels) #train data, test data input

kNN initialized..
(60000, 784)
(60000, 1)


In [ ]:
y_est = model.predict(te_images, k=3, p=2)
#이걸 기반으로 debugging해보고, 추가적으로, 가장 적합한 hyper paramter 찾기

(100, 784)


  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
# compute and print your kNN classification accuracy
print((y_est==te_labels).sum()/len(te_labels)) #y_est==te_labels -> 1, y_est!=te_labels ->0 -> .sum()으로 같은 녀석의 개수 구하기. (0.95~0.99 정도 나올 것)

0.99


In [ ]:
print(y_est)

[6 9 3 8 9 3 9 7 5 6 1 1 4 0 4 5 7 0 1 5 3 9 5 6 5 6 1 2 8 9 0 2 5 6 7 0 5
 1 1 5 0 9 5 2 9 3 1 3 4 4 7 9 6 0 5 9 3 9 6 8 1 4 2 5 5 9 0 5 4 7 7 4 5 1
 2 1 4 6 3 0 9 9 6 5 5 0 1 5 8 9 0 8 2 0 1 0 6 4 3 5]


In [ ]:
print(te_labels)

[6 9 8 8 9 3 9 7 5 6 1 1 4 0 4 5 7 0 1 5 3 9 5 6 5 6 1 2 8 9 0 2 5 6 7 0 0
 1 1 5 0 7 5 2 9 3 1 3 4 4 7 9 6 0 5 9 3 9 6 8 1 4 2 8 5 9 0 5 4 7 7 4 5 1
 2 1 4 6 3 0 9 9 6 5 5 0 1 5 8 9 0 8 2 0 1 0 5 4 3 5]
